In [1]:
from instruction import keypair_from_json
from processor import mint_nft
from solana.keypair import Keypair
from solana.rpc.api import Client
from state import ClassEnum
client = Client("https://api.devnet.solana.com")
print("Client is connected" if client.is_connected() else "Client is Disconnected")
payer_keypair = keypair_from_json("./keypair.json")
mint_keypair = Keypair()
mint_nft(payer_keypair, mint_keypair, ClassEnum.enum.Benitoite(), client)

Client is connected


FileNotFoundError: [Errno 2] No such file or directory: './keypair.json'